## Plot the standard deviation across the HH, HV, HHHV and RVI polarisations/metrics
- use to show which one has the highest variance over time, and therefore most useful for mapping wetland water level changes. 
- Figures 10 and 13 from the paper are output here

In [ ]:
import sys
import pandas as pd
import xarray as xr
import numpy as np
xr.set_options(cmap_sequential='jet')
import matplotlib.pyplot as plt
import warnings
import netCDF4
import datetime
import plotly
import plotly.express as px
import plotly.graph_objects as go
from affine import Affine
import cartopy.crs as ccrs
import nco
import xesmf as xe
from calendar import monthrange
import pickle as pkl
from datetime import date, datetime, timedelta
import math
import seaborn as sns
from scipy import stats
from IPython.display import display

plt.rcParams['figure.dpi'] = 1000
%xmode Minimal

In [ ]:
def plt_var(ds,var_name1, var_name2, title, filename,vmin,vmax):
    sns.set(font_scale=1.2, rc={'axes.facecolor':'white'}, style='ticks')
    ds.rename({var_name1:var_name2})[var_name2].plot(cmap ='cividis', vmin=vmin,vmax=vmax, size=4.5, aspect=1.4)
    plt.xlabel('')
    plt.xticks([16.5,17,17.5,18,18.5,19,19.5])
    plt.ylabel('')
    plt.title(title, fontsize=14)
    plt.show()
    plt.savefig(images + filename, bbox_inches='tight')

In [ ]:
# Define input data location here:
#ALOS_OUT = 

# Location to save images to:
#images = 

In [ ]:
var_names = ['HH','HV','HHHV','RVI']

# Reading in the SAR data polarisations
HH = xr.open_dataset(ALOS_OUT + 'ALOS_HH_area2.nc')
HV = xr.open_dataset(ALOS_OUT + 'ALOS_HV_area2.nc')
HHHV = xr.open_dataset(ALOS_OUT + 'ALOS_HHHV_ratio_area2.nc')
RVI = xr.open_dataset(ALOS_OUT + 'ALOS_RVI_area2.nc')

# land type map for masking the data for peatland pixels only
lt_map = xr.open_dataset(ALOS_OUT + 'landtype_100m.nc')


In [ ]:
%%time
# calculating the standard deviation maps over time

HH_stdev = HH.std(dim='time')
HV_stdev = HV.std(dim='time')
HHHV_stdev = HHHV.std(dim='time')
RVI_stdev = RVI.std(dim='time')

In [ ]:
# masking for peatland areas only
HH_stdev = HH_stdev.where(lt_map['type'].isin([4,5]))
HV_stdev = HV_stdev.where(lt_map['type'].isin([4,5]))
HHHV_stdev = HHHV_stdev.where(lt_map['type'].isin([4,5]))
RVI_stdev = RVI_stdev.where(lt_map['type'].isin([4,5]))

In [ ]:
plt_var(HH_stdev,'db','HH standard deviation (dB)', 'HH standard deviation','HH_stdev.png',0, 1.5)

In [ ]:
plt_var(HV_stdev,'db','HV standard deviation (dB)', 'HV standard deviation','HV_stdev.png',0,1.5)

In [ ]:
plt_var(HHHV_stdev,'db','HHHV standard deviation (dB)', 'HHHV standard deviation','HHHV_stdev.png',0,1.5)

In [ ]:
HH_mean = HH_mean.where(lt_map['type'].isin([4,5]))
HV_mean = HV_mean.where(lt_map['type'].isin([4,5]))
HHHV_mean = HHHV_mean.where(lt_map['type'].isin([4,5]))

In [ ]:
# masking for water
HH_mean = HH_mean.where(HH_mean['db']>-11)

In [ ]:
# HV mean values for both hardwood (HWS) and palm swamp (PS)
# are the two peaks due to the different vegetation types?
# HWS has two peaks, one lower than PS. 
# PS only has one peak
# indicative of differences in types of HWS - could these differences be used to improve the vegetation map?
# The spread for HWS seems large
HV_mean = HV_mean.where(HV_mean['db']>-50)
HV_mean['db'].plot.hist(bins=300,xlim=(-16,-10));

In [ ]:
# Palm swamp only
HV_mean_ps = HV_mean.where(lt_map['type']==4)
HV_mean_ps['db'].plot.hist(bins=300, xlim=(-16,-10));

In [ ]:
# hardwood swamp only 
HV_mean_hws = HV_mean.where(lt_map['type']==5)
HV_mean_hws['db'].plot.hist(bins=300, xlim=(-16,-10));

In [ ]:
# HH for palm swamp
HH_mean_ps = HH_mean.where(lt_map['type']==4)
HH_mean_ps['db'].plot.hist(bins=300, xlim=(-8.5,-3.5));

In [ ]:
# HH for hardwood swamp
HH_mean_hws = HH_mean.where(lt_map['type']==5)
HH_mean_hws['db'].plot.hist(bins=300, xlim=(-8.5,-3.5));

In [ ]:
HH_mean_all = xr.concat([HH_mean_ps,HH_mean_hws], dim='landtype').assign_coords(landtype = ['Palm swamp', 'Hardwood swamp'])
HH_mean_all

In [ ]:
df = HH_mean_all['db'].to_dataframe().reset_index()

In [ ]:
df = df.rename(columns={'landtype':'Land type','db':'HH backscatter (dB)'})
df

In [ ]:
sns.color_palette('colorblind',2)

In [ ]:
%%time

# a histogram plot comparing the HH backscatter between palm and hardwood swamp
fig, ax = plt.subplots()
sns.histplot(data=df, x='HH backscatter (dB)',hue='Land type', palette='colorblind',ax=ax)
ax.set_xlim(-8,-3.5)

# setting the y axis format to include a comma between the thousands for the count
plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
#plt.show()
Images = '/home/s0677837/Datastore/PHD/ALOS2/Palmer_folder/Outputs/Images/'
plt.savefig(Images + 'swamp_hh_hist.png', bbox_inches='tight')

In [ ]:
ps = df.loc[df['Land type'] == 'Palm swamp']
hws = df.loc[df['Land type'] == 'Hardwood swamp']
sns.distplot(ps['HH backscatter (dB)'], hist=True,rug=False)
sns.distplot(hws['HH backscatter (dB)'], hist=True,rug=False)